Section 1: Import Libraries

In [2]:
import requests # This libraries helps us to fetch data from API
import pandas as pd # For handling and analysing data
import numpy as np # For numerical operations
from sklearn.model_selection import train_test_split # To split data into training and testing sets
from sklearn.preprocessing import LabelEncoder # To convert categorical data into numerical values
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor # Models for classification and regression tasks
from sklearn.metrics import mean_squared_error # To measure the accuracy of our predictons
from datetime import datetime, timedelta #To handle date and time
import pytz

In [3]:
API_KEY = '0cfbe6840e85dae64a9fd8ad1abcb3e5' 
BASE_URL = 'https://api.openweathermap.org/data/2.5/'

1. Fetch Current Weather Data

In [4]:
def get_current_weather(city):
    url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()
    return {
        'city': data['name'],
        'current_temp': round(data['main']['temp'], 1),
        'feels_like': round(data['main']['feels_like'], 1),
        'temp_min': round(data['main']['temp_min'], 1),
        'temp_max': round(data['main']['temp_max'], 1),
        'humidity': round(data['main']['humidity'], 1),
        'description': data['weather'][0]['description'],
        'country': data['sys']['country'],
        'wind_gust_dir': data['wind']['deg'],
        'pressure': data['main']['pressure'],
        'Wind_Gust_Speed': data['wind']['speed']
    }

2. Read Historical Data

In [5]:
def read_historical_data(filename):
    df = pd.read_csv(filename) 
    df = df.dropna() #remove rows with missing values
    df = df.drop_duplicates()
    return df

3. Prepare data for training

In [6]:
def prepare_data(data):
    le = LabelEncoder() # Create a LabelEncoder instance
    data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
    data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])

    # Define the feature variable and target variables
    x = data[['MinTemp', 'MaxTemp','WindGustDir', 'WindGustSpeed', 'Humidity', 'Pressure', 'Temp']] #Feature Variable
    y = data['RainTomorrow']
    return x, y, le
    

4. Train Rain Prediction Model

In [7]:
def train_rain_model(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)
    print("Mean Squared Error For Rain Model")

    print(mean_squared_error(y_test, y_pred))

    return model

5. Prepare Regression Data

In [8]:
def prepare_regression_data(data, feature):
    x, y = [], [] # Initialize list for feature and target values

    for i in range(len(data) - 1):
        x.append(data[feature].iloc[i])

        y.append(data[feature].iloc[i+1])
    x = np.array(x).reshape(-1,1)
    y = np.array(y)
    return x,y

6. Train Regression Model

In [9]:
def train_regression_model(x, y):
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(x,y)
    return model

7. Predict Future

In [10]:
def predict_future(model, current_value):
    predictions = [current_value]

    for i in range(5):
        next_value = model.predict(np.array([[predictions[-1]]]))
        predictions.append(next_value[0])
    return predictions[1:]

8. Weather Analysis

In [17]:
def weather_view():
    city = input("Enter City Name: ").strip()
    current_weather = get_current_weather(city)

    # Load historical data
    historical_data = read_historical_data('weather.csv')  # Ensure this file exists in the same directory

    # Prepare and train the rain prediction model
    x, y, le = prepare_data(historical_data)
    rain_model = train_rain_model(x, y)

    # Map wind direction to compass points
    wind_deg = current_weather['wind_gust_dir'] % 360
    compass_points = [
        ("N", 0, 11.25), ("NNE", 11.25, 33.75), ("NE", 33.75, 56.25),
        ("ENE", 56.25, 78.75), ("E", 78.75, 101.25), ("ESE", 101.25, 123.75),
        ("SE", 123.75, 146.25), ("SSE", 146.25, 168.75), ("S", 168.75, 191.25),
        ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
        ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25),
        ("NNW", 326.25, 348.75), ("N", 348.75, 360)
    ]
    compass_direction = next(point for point, start, end in compass_points if start <= wind_deg < end)
    compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1

    # Build current data input for prediction
    current_data = {
        'MinTemp': current_weather['temp_min'],
        'MaxTemp': current_weather['temp_max'],
        'WindGustDir': compass_direction_encoded,
        'WindGustSpeed': current_weather['Wind_Gust_Speed'],
        'Humidity': current_weather['humidity'],
        'Pressure': current_weather['pressure'],
        'Temp': current_weather['current_temp'],
    }

    current_df = pd.DataFrame([current_data])

    # Predict rain
    rain_prediction = rain_model.predict(current_df)[0]

    # Prepare regression models for temperature and humidity
    x_temp, y_temp = prepare_regression_data(historical_data, 'Temp')
    x_hum, y_hum = prepare_regression_data(historical_data, 'Humidity')

    temp_model = train_regression_model(x_temp, y_temp)
    hum_model = train_regression_model(x_hum, y_hum)

    # Predict future temperature and humidity
    future_temp = predict_future(temp_model, current_weather['temp_min'])
    future_humidity = predict_future(hum_model, current_weather['humidity'])

    # Generate future time labels
    timezone = pytz.timezone('Asia/Kolkata')
    now = datetime.now(timezone)
    next_hour = now + timedelta(hours=1)
    next_hour = next_hour.replace(minute=0, second=0, microsecond=0)
    future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

    # Display the results
    print(f"\nCity: {city}, {current_weather['country']}")
    print(f"Current Temperature: {current_weather['current_temp']}°C")
    print(f"Feels Like: {current_weather['feels_like']}°C")
    print(f"Minimum Temperature: {current_weather['temp_min']}°C")
    print(f"Maximum Temperature: {current_weather['temp_max']}°C")
    print(f"Humidity: {current_weather['humidity']}%")
    print(f"Weather Description: {current_weather['description']}")
    print(f"Rain Prediction: {'Yes' if rain_prediction else 'No'}")

    print("\nFuture Temperature Predictions:")
    for time, temp in zip(future_times, future_temp):
        print(f"{time}: {round(temp, 1)}°C")

    print("\nFuture Humidity Predictions:")
    for time, humidity in zip(future_times, future_humidity):
        print(f"{time}: {round(humidity, 1)}%")


# Run the function
weather_view()


Enter City Name:  Bangalore


Mean Squared Error For Rain Model
0.1506849315068493

City: Bangalore, IN
Current Temperature: 25.2°C
Feels Like: 25.4°C
Minimum Temperature: 25.2°C
Maximum Temperature: 25.2°C
Humidity: 62%
Weather Description: overcast clouds
Rain Prediction: Yes

Future Temperature Predictions:
14:00: 20.1°C
15:00: 21.9°C
16:00: 21.3°C
17:00: 20.5°C
18:00: 22.2°C

Future Humidity Predictions:
14:00: 56.5%
15:00: 40.7%
16:00: 52.9%
17:00: 51.0%
18:00: 46.6%
